### **Balanced Risk Set Matching in Observational Studies**
---
**Authors:**  
📌 *Justin Chris Catingub*  
📌 *Juliana Marie Ochea*  
---
    
## **Data Analytics Assignment**

### **Introduction**

In observational studies, treatment is often not randomly assigned, making causal inference difficult.
To reduce bias, we use Balanced Risk Set Matching (BRSM), a technique that pairs treated individuals
with similar controls.

This notebook implements BRSM using Mahalanobis Distance Matching, based on the journal 'Balanced Risk Set Matching'.


# **Step 1: Import Necessary Libraries**

In [18]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import mahalanobis
from scipy.optimize import linear_sum_assignment
from scipy.stats import wilcoxon

# **Step 2: Define the Balanced Risk Set Matching Class**

In [19]:
class BalancedRiskSetMatching:
    def __init__(self, data, treatment_col, time_col, covariates):
        """Initialize with dataset, treatment indicator, time column, and covariates."""
        self.data = data.copy()
        self.treatment_col = treatment_col
        self.time_col = time_col
        self.covariates = covariates

    def compute_mahalanobis_matrix(self, treated, controls):
        """Compute Mahalanobis distances between treated and control patients."""
        try:
            cov_matrix = np.cov(self.data[self.covariates].dropna().T)
            cov_matrix += np.eye(cov_matrix.shape[0]) * 1e-6
            inv_cov = np.linalg.inv(cov_matrix)
        except np.linalg.LinAlgError:
            raise ValueError("Covariance matrix is singular. Check for collinear features or insufficient data.")
        
        dist_matrix = np.zeros((len(treated), len(controls)))
        for i, t in enumerate(treated[self.covariates].values):
            for j, c in enumerate(controls[self.covariates].values):
                dist_matrix[i, j] = mahalanobis(t, c, inv_cov)
        
        return dist_matrix
    
    def match_pairs(self):
        """Perform optimal risk set matching using Mahalanobis distance."""
        treated = self.data[self.data[self.treatment_col] == 1]
        controls = self.data[self.data[self.treatment_col] == 0]
        
        if treated.empty or controls.empty:
            raise ValueError("Insufficient treated or control subjects for matching.")
        
        dist_matrix = self.compute_mahalanobis_matrix(treated, controls)
        row_ind, col_ind = linear_sum_assignment(dist_matrix)
        
        matched_pairs = pd.DataFrame({
            'treated_id': treated.iloc[row_ind].index,
            'control_id': controls.iloc[col_ind].index,
            'distance': dist_matrix[row_ind, col_ind]
        })
        
        return matched_pairs
    
    def perform_wilcoxon_test(self, outcome_col):
        """Conduct Wilcoxon signed-rank test on matched pairs."""
        matched_pairs = self.match_pairs()
        treated_outcomes = self.data.loc[matched_pairs['treated_id'], outcome_col].values
        control_outcomes = self.data.loc[matched_pairs['control_id'], outcome_col].values
        
        if len(treated_outcomes) < 10:
            raise ValueError("Not enough matched pairs for Wilcoxon test. Consider increasing sample size.")
        
        stat, p_value = wilcoxon(treated_outcomes, control_outcomes)
        return stat, p_value


# **Step 3: Load Dataset** 

We begin by loading the dataset containing 50 observations.
Each row represents a patient with covariates such as pain, urgency, and frequency.

In [20]:
df = pd.read_csv("data.csv")

We ensure the data set exists in the same directory

In [21]:
import os
print(os.listdir())

['.git', '.gitattributes', 'A1_test.ipynb', 'data.csv']


# **Step 4: Initialize the Matching Class**

We now initialize the BalancedRiskSetMatching class with our dataset,
setting 'treated' as the treatment column and 'pain', 'urgency', and 'frequency' as covariates.


In [22]:
brsm = BalancedRiskSetMatching(df, treatment_col='treated', time_col='time', covariates=['pain', 'urgency', 'frequency'])

# **Step 5: Perform Matching**

We apply the matching process to find optimal treated-control pairs using Mahalanobis distance.


In [23]:
matched_pairs = brsm.match_pairs()
print("Matched Pairs:\n", matched_pairs)

Matched Pairs:
     treated_id  control_id  distance
0            0          27  0.000000
1            1          25  0.000000
2            2          42  2.545835
3            3          26  0.000000
4            4          37  3.037686
5            5          30  0.848612
6            6          31  0.848612
7            7          45  0.848612
8            8          36  0.000000
9            9          38  0.000000
10          10          35  0.848612
11          11          40  2.545835
12          12          28  0.000000
13          13          49  2.545835
14          14          47  0.000000
15          15          33  0.000000
16          16          44  0.000000
17          17          29  0.000000
18          18          41  0.000000
19          19          43  2.545835
20          20          32  0.848612
21          21          46  0.848612
22          22          39  1.697224
23          23          48  0.848612
24          24          34  0.000000


# **Step 6: Perform Wilcoxon Signed-Rank Test**

To check if there is a significant difference in nocturnal frequency between the treated and control groups,
we perform the Wilcoxon Signed-Rank Test.


In [24]:
stat, p_value = brsm.perform_wilcoxon_test('nocturnal_frequency')
print(f"Wilcoxon Test Statistic: {stat}, p-value: {p_value}")   

Wilcoxon Test Statistic: 13.0, p-value: 0.001718038567391757


# **Step 7: Conclusion & Insights**

Results from the Wilcoxon test help determine whether treatment significantly impacts nocturnal frequency.
Interpretation:
- If p < 0.05, there is a significant difference between the groups.
- If p > 0.05, the effect of treatment is not statistically significant.

